In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import copy

from psm_utils.psm import PSM
from psm_utils.psm_list import PSMList
from psm_utils.io import write_file
from deeplc import FeatExtractor


from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras import callbacks
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from IPython.display import display
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Dense, Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Bidirectional, LSTM
from sklearn.metrics import mean_squared_error

In [ ]:
#encoding by atomic composition 

infile = pd.read_csv("/home/emmy/Notebooks2/MQ_alignment_output/evidence_aligned_6.csv") #hier eigen aligned csv file aanroepen
infile.head()
psm_list = [] #psm_list is type object 
for idx,row in infile.iterrows():
    seq = row["Sequence"]
    charge = row["Charge"]  

    peptidoform = f"{seq}/{charge}"

    psm_list.append(PSM(peptidoform=peptidoform,spectrum_id=idx))
    
psm_list = PSMList(psm_list=psm_list)

feat_extractor = FeatExtractor()
matrices = feat_extractor.encode_atoms(psm_list, list(range(len(psm_list))), predict_ccs=True)

In [3]:
data = pd.read_csv("/home/emmy/Notebooks2/MQ_alignment_output/evidence_aligned_6.csv") #reading in the data
ccs_df = data[['CCS']]
ccs_df.head()

,CCS
0,409.181586
1,675.752631
2,593.085242
3,682.043900
4,492.912110


In [4]:
np.stack(list(matrices["matrix"].values())).shape

(1148760, 60, 6)

In [5]:
np.stack(list(matrices["matrix_all"].values())).shape

(1148760, 12)

In [6]:
matrix = np.stack(list(matrices["matrix"].values()))
matrix_all = np.stack(list(matrices["matrix_all"].values()))

In [8]:
# Split the data into training and testing sets
matrix_train, matrix_test, matrix_all_train, matrix_all_test, ccs_train, ccs_test = train_test_split(matrix, matrix_all, ccs_df, test_size=0.2, random_state=42)

In [9]:
# Further split the training data into training and validation sets if needed
matrix_train, matrix_val, matrix_all_train, matrix_all_val, ccs_train, ccs_val = train_test_split(matrix_train, matrix_all_train, ccs_train, test_size=0.1, random_state=42)

In [23]:
#checking of the correct ccs value is coupled to the correct peptide sequence

if len(matrix_all_train) > 0 and len(ccs_train) > 0:
    print("Train set:")
    for i in range(5):
        print("Peptide sequence:", matrix_all_train[i], "CCS value:", ccs_train['CCS'].iloc[i])
else:
    print("Empty datasets")


Train set:
Peptide sequence: [ 96 148  24  33   0   0  20   0   0   0   0   3] CCS value: 542.3662
Peptide sequence: [46 69 11 10  0  0  8  0  0  0  0  2] CCS value: 336.038086
Peptide sequence: [57 96 14 18  0  0 11  0  0  0  0  2] CCS value: 375.9927733333333
Peptide sequence: [ 84 137  23  32   1   0  19   0   0   0   0   2] CCS value: 473.0402284615385
Peptide sequence: [ 67 108  18  23   1   0  15   0   0   0   0   2] CCS value: 421.12067


In [58]:
# Define the model
# Assuming your input shapes
matrix_shape = (1148760, 60, 6)
matrix_all_shape = (1148760, 12)

# Define input layers for each matrix
input_matrix = Input(shape=(matrix_shape[1], matrix_shape[2]), name='matrix_input')
input_matrix_all = Input(shape=(matrix_all_shape[1],), name='matrix_input_all')

# Flatten the input matrices
flattened_matrix = Flatten()(input_matrix)
flattened_matrix_all = input_matrix_all  # No need to flatten

# Concatenate the flattened and non-flattened outputs
concatenated_outputs = Concatenate()([flattened_matrix, flattened_matrix_all])


dense1 = Dense(units=512, activation = "relu")(concatenated_outputs)
dense2 = Dense(units=512, activation = "relu")(dense1)
dense3 = Dense(units=512, activation = "relu")(dense2)
output = Dense(units=1)(dense3)

# Create the model
model = Model(inputs=[input_matrix, input_matrix_all], outputs=output)


# Compile the model

model.compile(
    optimizer='adam',
    loss='mae'
)
# Train the model with your input and output data
history = model.fit([matrix_train, matrix_all_train], ccs_train, epochs=10, batch_size=256, validation_data=([matrix_val, matrix_all_val], ccs_val))


# convert the training history to a dataframe
history_df_2 = pd.DataFrame(history.history)




Epoch 1/10
3231/3231 [==============================] - 33s 10ms/step - loss: 19.5854 - val_loss: 13.4270
Epoch 2/10
3231/3231 [==============================] - 33s 10ms/step - loss: 14.3132 - val_loss: 13.5223
Epoch 3/10
3231/3231 [==============================] - 24s 8ms/step - loss: 13.9334 - val_loss: 14.3406
Epoch 4/10
3231/3231 [==============================] - 21s 7ms/step - loss: 13.4897 - val_loss: 16.4553
Epoch 5/10
3231/3231 [==============================] - 21s 7ms/step - loss: 13.2754 - val_loss: 12.5026
Epoch 6/10
3231/3231 [==============================] - 21s 6ms/step - loss: 13.0308 - val_loss: 15.0347
Epoch 7/10
3231/3231 [==============================] - 21s 6ms/step - loss: 12.9214 - val_loss: 12.1725
Epoch 8/10
3231/3231 [==============================] - 21s 6ms/step - loss: 12.7763 - val_loss: 12.3239
Epoch 9/10
3231/3231 [==============================] - 21s 6ms/step - loss: 12.7292 - val_loss: 12.2417
Epoch 10/10
3231/3231 [==============================

In [60]:
print("Minimum validation loss: {}".format(history_df_2['val_loss'].min()))

Minimum validation loss: 12.172513008117676


In [61]:
history_df_2.to_csv('history_2.csv', index=False) #save the history to a csv file for visualization

In [56]:
#add early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Define the model
# Assuming your input shapes
matrix_shape = (1148760, 60, 6)
matrix_all_shape = (1148760, 12)

# Define input layers for each matrix
input_matrix = Input(shape=(matrix_shape[1], matrix_shape[2]), name='matrix_input')
input_matrix_all = Input(shape=(matrix_all_shape[1],), name='matrix_input_all')

# Flatten the input matrices
flattened_matrix = Flatten()(input_matrix)
flattened_matrix_all = input_matrix_all  # No need to flatten

# Concatenate the flattened and non-flattened outputs
concatenated_outputs = Concatenate()([flattened_matrix, flattened_matrix_all])


dense1 = Dense(units=512, activation = "relu")(concatenated_outputs)
dense2 = Dense(units=512, activation = "relu")(dense1)
dense3 = Dense(units=512, activation = "relu")(dense2)
output = Dense(units=1)(dense3)

# Create the model
model2 = Model(inputs=[input_matrix, input_matrix_all], outputs=output)


# Compile the model

model2.compile(
    optimizer='adam',
    loss='mae'
)
# Train the model with your input and output data
history = model2.fit([matrix_train, matrix_all_train], ccs_train, epochs=500, batch_size=256, validation_data=([matrix_val, matrix_all_val], ccs_val), callbacks=[early_stopping])

history_df = pd.DataFrame(history.history)
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))


Epoch 1/500
 125/3231 [>.............................] - ETA: 32s - loss: 64.8568

3231/3231 [==============================] - 35s 11ms/step - loss: 19.5496 - val_loss: 15.8622
Epoch 2/500
3231/3231 [==============================] - 35s 11ms/step - loss: 14.3577 - val_loss: 13.0160
Epoch 3/500
3231/3231 [==============================] - 34s 11ms/step - loss: 13.7708 - val_loss: 12.7879
Epoch 4/500
3231/3231 [==============================] - 34s 11ms/step - loss: 13.4237 - val_loss: 12.9888
Epoch 5/500
3231/3231 [==============================] - 34s 11ms/step - loss: 13.2281 - val_loss: 12.7121
Epoch 6/500
3231/3231 [==============================] - 34s 10ms/step - loss: 13.0719 - val_loss: 12.4303
Epoch 7/500
3231/3231 [==============================] - 34s 10ms/step - loss: 12.9920 - val_loss: 12.3545
Epoch 8/500
3231/3231 [==============================] - 34s 11ms/step - loss: 12.9075 - val_loss: 12.4955
Epoch 9/500
3231/3231 [==============================] - 34s 11ms/step - loss: 12.7433 - val_loss: 12.6163
Epoch 10/500
3231/3231 [=========================

In [57]:
history_df.to_csv('history.csv', index=False) #save the history to a csv file for visualization

In [64]:
# Predict CCS values test set
ccs_test["Model2_predictions"] = model2.predict((matrix_test, matrix_all_test))

7180/7180 [==============================] - 5s 745us/step


In [65]:
ccs_test.head()

,CCS,Model2_predictions
129919,420.248133,427.884583
1129049,441.017200,430.113831
161312,550.032030,566.157043
15185,501.311247,500.421722
939427,520.231105,494.102081


In [66]:
ccs_test.to_csv('ccs_test_model2.csv', index=False) #save the test set to a csv file for visualization

In [69]:
#add early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)


# Define the model with more layers and neurons
# Assuming your input shapes
matrix_shape = (1148760, 60, 6)
matrix_all_shape = (1148760, 12)

# Define input layers for each matrix
input_matrix = Input(shape=(matrix_shape[1], matrix_shape[2]), name='matrix_input')
input_matrix_all = Input(shape=(matrix_all_shape[1],), name='matrix_input_all')

# Flatten the input matrices
flattened_matrix = Flatten()(input_matrix)
flattened_matrix_all = input_matrix_all  # No need to flatten

# Concatenate the flattened and non-flattened outputs
concatenated_outputs = Concatenate()([flattened_matrix, flattened_matrix_all])


dense1 = Dense(units=1024, activation = "relu")(concatenated_outputs)
dense2 = Dense(units=1014, activation = "relu")(dense1)
dense3 = Dense(units=1024, activation = "relu")(dense2)
dense4 = Dense(units=512, activation = "relu")(dense3)
output = Dense(units=1)(dense4)

# Create the model
model3 = Model(inputs=[input_matrix, input_matrix_all], outputs=output)


# Compile the model

model3.compile(
    optimizer='adam',
    loss='mae'
)
# Train the model with your input and output data
history = model3.fit([matrix_train, matrix_all_train], ccs_train, epochs=500, batch_size=256, validation_data=([matrix_val, matrix_all_val], ccs_val), callbacks=[early_stopping])

history_df_3 = pd.DataFrame(history.history)
print("Minimum validation loss: {}".format(history_df_3['val_loss'].min()))

Epoch 1/500
3231/3231 [==============================] - 40s 12ms/step - loss: 20.3715 - val_loss: 14.0726
Epoch 2/500
3231/3231 [==============================] - 40s 12ms/step - loss: 15.0046 - val_loss: 13.5355
Epoch 3/500
3231/3231 [==============================] - 36s 11ms/step - loss: 14.3380 - val_loss: 13.4943
Epoch 4/500
3231/3231 [==============================] - 36s 11ms/step - loss: 13.7926 - val_loss: 14.5801
Epoch 5/500
3231/3231 [==============================] - 40s 12ms/step - loss: 13.5000 - val_loss: 12.4856
Epoch 6/500
3231/3231 [==============================] - 39s 12ms/step - loss: 13.2981 - val_loss: 12.2679
Epoch 7/500
3231/3231 [==============================] - 40s 12ms/step - loss: 13.1088 - val_loss: 14.7322
Epoch 8/500
3231/3231 [==============================] - 40s 12ms/step - loss: 12.9788 - val_loss: 13.0394
Epoch 9/500
3231/3231 [==============================] - 40s 12ms/step - loss: 12.9095 - val_loss: 12.0974
Epoch 10/500
3231/3231 [=============

In [72]:
history_df_3.to_csv('history_3.csv', index=False) #save the history to a csv file for visualization
# Predict CCS values test set
ccs_test["Model3_predictions"] = model3.predict((matrix_test, matrix_all_test))

7180/7180 [==============================] - 7s 966us/step


In [76]:
ccs_test.head()
ccs_test.to_csv('ccs_test_model3.csv', index=False) #save the test set to a csv file for visualization

In [77]:
#adding dropout layers

#add early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Define the model
# Assuming your input shapes
matrix_shape = (1148760, 60, 6)
matrix_all_shape = (1148760, 12)

# Define input layers for each matrix
input_matrix = Input(shape=(matrix_shape[1], matrix_shape[2]), name='matrix_input')
input_matrix_all = Input(shape=(matrix_all_shape[1],), name='matrix_input_all')

# Flatten the input matrices
flattened_matrix = Flatten()(input_matrix)
flattened_matrix_all = input_matrix_all  # No need to flatten

# Concatenate the flattened and non-flattened outputs
concatenated_outputs = Concatenate()([flattened_matrix, flattened_matrix_all])


dense1 = Dense(units=1024, activation = "relu")(concatenated_outputs)
dropout1 = Dropout(0.3)(dense1) 
dense2 = Dense(units=1014, activation = "relu")(dense1)
dropout2 = Dropout(0.3)(dense2) 
dense3 = Dense(units=1024, activation = "relu")(dense2)
dropout3 = Dropout(0.3)(dense3)
dense4 = Dense(units=512, activation = "relu")(dense3)
dropout4 = Dropout(0.3)(dense4) 
output = Dense(units=1)(dropout4)

# Create the model
model4 = Model(inputs=[input_matrix, input_matrix_all], outputs=output)


# Compile the model

model4.compile(
    optimizer='adam',
    loss='mae'
)
# Train the model with your input and output data
history = model4.fit([matrix_train, matrix_all_train], ccs_train, epochs=500, batch_size=256, validation_data=([matrix_val, matrix_all_val], ccs_val), callbacks=[early_stopping])

history_df_4 = pd.DataFrame(history.history)
print("Minimum validation loss: {}".format(history_df_4['val_loss'].min()))

Epoch 1/500
3231/3231 [==============================] - 30s 9ms/step - loss: 29.1184 - val_loss: 16.0350
Epoch 2/500
3231/3231 [==============================] - 30s 9ms/step - loss: 24.8793 - val_loss: 20.2683
Epoch 3/500
3231/3231 [==============================] - 30s 9ms/step - loss: 24.4736 - val_loss: 20.8097
Epoch 4/500
3231/3231 [==============================] - 30s 9ms/step - loss: 24.1170 - val_loss: 13.8475
Epoch 5/500
3231/3231 [==============================] - 30s 9ms/step - loss: 23.8298 - val_loss: 13.7631
Epoch 6/500
3231/3231 [==============================] - 30s 9ms/step - loss: 23.6693 - val_loss: 15.0453
Epoch 7/500
3231/3231 [==============================] - 30s 9ms/step - loss: 23.5504 - val_loss: 15.3630
Epoch 8/500
3231/3231 [==============================] - 30s 9ms/step - loss: 23.4855 - val_loss: 13.4836
Epoch 9/500
3231/3231 [==============================] - 30s 9ms/step - loss: 23.3947 - val_loss: 13.7568
Epoch 10/500
3231/3231 [======================

In [79]:
history_df_4.to_csv('history_4.csv', index=False) #save the history to a csv file for visualization
# Predict CCS values test set
ccs_test["Model4_predictions"] = model4.predict((matrix_test, matrix_all_test))

7180/7180 [==============================] - 11s 2ms/step


In [81]:
ccs_test.head()
ccs_test.to_csv('ccs_test_model4.csv', index=False) #save the test set to a csv file for visualization

,CCS,Model2_predictions,Model3_predictions,Model4_predictions
129919,420.248133,427.884583,423.535065,423.233276
1129049,441.017200,430.113831,429.712219,431.340668
161312,550.032030,566.157043,564.284912,566.298706
15185,501.311247,500.421722,505.957703,503.284088
939427,520.231105,494.102081,493.030487,487.198242


In [82]:
#adding batch normalization

# Define the model
# Assuming your input shapes
matrix_shape = (1148760, 60, 6)
matrix_all_shape = (1148760, 12)

# Define input layers for each matrix
input_matrix = Input(shape=(matrix_shape[1], matrix_shape[2]), name='matrix_input')
input_matrix_all = Input(shape=(matrix_all_shape[1],), name='matrix_input_all')

# Flatten the input matrices
flattened_matrix = Flatten()(input_matrix)
flattened_matrix_all = input_matrix_all  # No need to flatten

# Concatenate the flattened and non-flattened outputs
concatenated_outputs = Concatenate()([flattened_matrix, flattened_matrix_all])

# Add dense layers with dropout and batch normalization
dense1 = Dense(units=1024, activation = "relu")(concatenated_outputs)
dropout1 = Dropout(0.3)(dense1)  # Add dropout layer with 50% dropout rate
batch_norm1 = BatchNormalization()(dropout1)  # Add batch normalization layer
dense2 = Dense(units=1014, activation = "relu")(batch_norm1)
dropout2 = Dropout(0.3)(dense2)  # Add dropout layer with 50% dropout rate
batch_norm2 = BatchNormalization()(dropout2)  # Add batch normalization layer
dense3 = Dense(units=1024, activation = "relu")(batch_norm2)
dropout3 = Dropout(0.3)(dense3)  # Add dropout layer with 50% dropout rate
batch_norm3 = BatchNormalization()(dropout3)  # Add batch normalization layer
dense4 = Dense(units=512, activation = "relu")(batch_norm3)
dropout4 = Dropout(0.3)(dense4)  # Add dropout layer with 50% dropout rate
batch_norm4 = BatchNormalization()(dropout4)  # Add batch normalization layer
output = Dense(units=1)(batch_norm4)

# Create the model
model5 = Model(inputs=[input_matrix, input_matrix_all], outputs=output)

# Compile the model
model5.compile(
    optimizer='adam',
    loss='mae'
)

# Train the model with your input and output data
history = model5.fit([matrix_train, matrix_all_train], ccs_train, epochs=500, batch_size=256, validation_data=([matrix_val, matrix_all_val], ccs_val), callbacks=[early_stopping])

history_df_5 = pd.DataFrame(history.history)
print("Minimum validation loss: {}".format(history_df_5['val_loss'].min()))

Epoch 1/500
3231/3231 [==============================] - 55s 17ms/step - loss: 77.0874 - val_loss: 14.8685
Epoch 2/500
3231/3231 [==============================] - 55s 17ms/step - loss: 17.3825 - val_loss: 15.6998
Epoch 3/500
3231/3231 [==============================] - 43s 13ms/step - loss: 16.5993 - val_loss: 27.9591
Epoch 4/500
3231/3231 [==============================] - 41s 13ms/step - loss: 16.2750 - val_loss: 22.5686
Epoch 5/500
3231/3231 [==============================] - 41s 13ms/step - loss: 15.9662 - val_loss: 25.2746
Epoch 6/500
3231/3231 [==============================] - 44s 14ms/step - loss: 15.7836 - val_loss: 25.1666
Epoch 7/500
3231/3231 [==============================] - 54s 17ms/step - loss: 15.7323 - val_loss: 25.5724
Epoch 8/500
3231/3231 [==============================] - 55s 17ms/step - loss: 15.5738 - val_loss: 25.5539
Epoch 9/500
3231/3231 [==============================] - 55s 17ms/step - loss: 15.5090 - val_loss: 26.2594
Epoch 10/500
3231/3231 [=============

In [83]:
history_df_5.to_csv('history_5.csv', index=False) #save the history to a csv file for visualization
# Predict CCS values test set
ccs_test["Model5_predictions"] = model5.predict((matrix_test, matrix_all_test))

7180/7180 [==============================] - 7s 1ms/step


In [87]:
ccs_test.head()
ccs_test.to_csv('ccs_test_model5.csv', index=False) #save the test set to a csv file for visualization

,CCS,Model2_predictions,Model3_predictions,Model4_predictions,Model5_predictions
129919,420.248133,427.884583,423.535065,423.233276,425.708954
1129049,441.017200,430.113831,429.712219,431.340668,443.349762
161312,550.032030,566.157043,564.284912,566.298706,549.194092
15185,501.311247,500.421722,505.957703,503.284088,517.718506
939427,520.231105,494.102081,493.030487,487.198242,480.236694


In [ ]:
%matplotlib inline

In [ ]:
ccs_test_model3 = pd.read_csv('/home/emmy/Notebooks2/output_FNN/ccs_test_model3.csv')



if len(ccs_df.index) < 1e4:
    set_alpha = 0.2
    set_size = 3
else:
    set_alpha = 0.05
    set_size = 1

# Scatter plot the observations on the test set against the predictions on the same set
plt.scatter(
    ccs_test_model3["CCS"],
    ccs_test_model3["Model3_predictions"],
    alpha=set_alpha,
    s=set_size,
)



# Plot a diagonal the points should be one
plt.plot([300,1100],[300,1100],c="grey")

legend = plt.legend()

for lh in legend.legendHandles:
    lh.set_sizes([25])
    lh.set_alpha(1)

# Get the predictions and calculate performance metrics
predictions = ccs_test_model3["Model3_predictions"]
true_ccs = ccs_test_model3["CCS"]
mare = round(sum((abs(predictions-true_ccs)/true_ccs)*100)/len(predictions),3)
pcc = round(pearsonr(predictions,true_ccs)[0],3)
perc_95 = round(np.percentile((abs(predictions-true_ccs)/true_ccs)*100,95)*2,2)

# Calculate MAE
mae = mean_absolute_error(true_ccs, predictions)

plt.title(f"FNN - PCC: {pcc} - MARE: {mare}% - 95th percentile: {perc_95}% - MAE: {mae}")

ax = plt.gca()
ax.set_aspect('equal')

plt.xlabel("Observed CCS (^2)")
plt.ylabel("Predicted CCS (^2)")

plt.show()